In [1]:
# import basic libraries
import yt
import pickle
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

# import halo catalogue func
from yt.analysis_modules.halo_analysis.api import *

In [3]:
import tempfile
import shutil
import os

# Create temporary directory for storing files
tmpdir = tempfile.mkdtemp()

In [54]:
# load dataset
ds = yt.load('~/../../tigress/cen/LG4_2048_40pc/RD0070/redshift0070')

yt : [INFO     ] 2017-06-29 13:54:45,977 Parameters: current_time              = 158.01757996843
yt : [INFO     ] 2017-06-29 13:54:45,979 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2017-06-29 13:54:45,983 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-29 13:54:45,986 Parameters: domain_right_edge         = [ 1.  1.  1.]
yt : [INFO     ] 2017-06-29 13:54:45,990 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-29 13:54:45,992 Parameters: current_redshift          = 4.9999999832278
yt : [INFO     ] 2017-06-29 13:54:45,994 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-06-29 13:54:45,996 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-06-29 13:54:45,999 Parameters: hubble_constant           = 0.6933


In [55]:
ds.print_stats()

Parsing Hierarchy : 100%|██████████| 44216/44216 [00:03<00:00, 13670.01it/s]
yt : [INFO     ] 2017-06-29 13:54:55,232 Gathering a field list (this may take a moment.)


level	# grids	       # cells	     # cells^3
----------------------------------------------
  0	   256	       2097152	           128
  1	    60	        547304	            82
  2	   187	       2071552	           128
  3	   898	       9903616	           215
  4	  5193	      55377920	           382
  5	 11014	      58419496	           389
  6	 10277	      39511880	           341
  7	  7393	      20631864	           275
  8	  4828	      10668936	           221
  9	  1558	       5499888	           177
 10	  1220	       5897424	           181
 11	  1332	       7614472	           197
----------------------------------------------
   	 44216	     218241504


t = 1.58017580e+02 = 3.75573046e+16 s = 1.19011917e+09 years

Smallest Cell:
	Width: 2.751e-05 Mpc
	Width: 2.751e+01 pc
	Width: 5.675e+06 AU
	Width: 8.489e+19 cm


In [17]:
ds.field_list

[('all', 'creation_time'),
 ('all', 'dynamical_time'),
 ('all', 'initial_mass'),
 ('all', 'metallicity_fraction'),
 ('all', 'particle_index'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_type'),
 ('all', 'particle_velocity_x'),
 ('all', 'particle_velocity_y'),
 ('all', 'particle_velocity_z'),
 ('all', 'typeia_fraction'),
 ('enzo', 'Dark_Matter_Density'),
 ('enzo', 'Density'),
 ('enzo', 'Electron_Density'),
 ('enzo', 'H2II_Density'),
 ('enzo', 'H2I_Density'),
 ('enzo', 'HII_Density'),
 ('enzo', 'HI_Density'),
 ('enzo', 'HM_Density'),
 ('enzo', 'HeIII_Density'),
 ('enzo', 'HeII_Density'),
 ('enzo', 'HeI_Density'),
 ('enzo', 'Metal_Density'),
 ('enzo', 'Temperature'),
 ('enzo', 'TotalEnergy'),
 ('enzo', 'x-velocity'),
 ('enzo', 'y-velocity'),
 ('enzo', 'z-velocity'),
 ('io', 'creation_time'),
 ('io', 'dynamical_time'),
 ('io', 'initial_mass'),
 ('io', 'metallicity_fraction'),
 ('io', 'part

In [4]:
# specify boundaries of zoom-in box
# scaling factor multiplied by info from text file 
# units in cm
scaling = 2.22535525e+25 # scales dataset coords to cm

xmin = scaling*0.39319589 * u.cm
ymin = scaling*0.42984636 * u.cm
zmin = scaling*0.41706725 * u.cm

xmax = scaling*0.56298484 * u.cm
ymax = scaling*0.55089246 * u.cm
zmax = scaling*0.56698254 * u.cm

In [5]:
# load halo dataset
halos_ds = yt.load('./LG4_2048_40pc/rd0070/halo_catalogs/catalog/catalog0070_thres160.0.h5')

yt : [INFO     ] 2017-06-29 17:07:14,396 Parameters: current_time              = 3.75573046409e+16
yt : [INFO     ] 2017-06-29 17:07:14,411 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-06-29 17:07:14,416 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-29 17:07:14,423 Parameters: domain_right_edge         = [  2.22535525e+25   2.22535525e+25   2.22535525e+25]
yt : [INFO     ] 2017-06-29 17:07:14,429 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-29 17:07:14,435 Parameters: current_redshift          = 4.99999998323
yt : [INFO     ] 2017-06-29 17:07:14,441 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-06-29 17:07:14,446 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-06-29 17:07:14,450 Parameters: hubble_constant           = 0.6933


In [6]:
halos_ds.field_list

yt : [INFO     ] 2017-06-29 17:07:17,244 Allocating for 9.178e+03 particles (index particle type 'all')
yt : [INFO     ] 2017-06-29 17:07:17,412 Identified 3.025e+03 octs


[('all', 'particle_identifier'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'virial_radius'),
 ('halos', 'particle_identifier'),
 ('halos', 'particle_mass'),
 ('halos', 'particle_position_x'),
 ('halos', 'particle_position_y'),
 ('halos', 'particle_position_z'),
 ('halos', 'virial_radius')]

In [49]:
halos_ds.derived_field_list

[('all', 'mesh_id'),
 ('all', 'particle_identifier'),
 ('all', 'particle_mass'),
 ('all', 'particle_ones'),
 ('all', 'particle_position'),
 ('all', 'particle_position_cylindrical_radius'),
 ('all', 'particle_position_cylindrical_theta'),
 ('all', 'particle_position_cylindrical_z'),
 ('all', 'particle_position_relative'),
 ('all', 'particle_position_relative_x'),
 ('all', 'particle_position_relative_y'),
 ('all', 'particle_position_relative_z'),
 ('all', 'particle_position_spherical_phi'),
 ('all', 'particle_position_spherical_radius'),
 ('all', 'particle_position_spherical_theta'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_radius'),
 ('all', 'particle_spherical_position_phi'),
 ('all', 'particle_spherical_position_radius'),
 ('all', 'particle_spherical_position_theta'),
 ('all', 'virial_radius'),
 ('deposit', 'all_cic'),
 ('deposit', 'all_count'),
 ('deposit', 'all_density'),
 ('deposit', 'all_mass'),
 ('deposi

In [7]:
# Instantiate a catalog using those two paramter files
hc = HaloCatalog(halos_ds=halos_ds, output_dir=os.path.join(tmpdir, 'halo_catalog'))
hc.load()

# create projection plot
p = yt.ProjectionPlot(sp, "x", "Dark_Matter_Density", width=(1.5, 'Mpc'))
p.set_zlim(field="Dark_Matter_Density", zmin=1e23, zmax=1e27)
#p.annotate_halos(hc, factor = "particle_mass")
p.annotate_halos(hc)
p.display()

In [133]:
hc.catalog.sort(key= lambda a : a['virial_radius'].to_ndarray())

In [8]:
satellite = []
nonsatellite = []

In [70]:
for halo1 in hc.halo_list:
    # find first index
    index1 = halo1.quantities.get('particle_identifier')
    
    # create bool to check if this halo isn't satellite
    isSatellite = False
    
    # find parameters of halo1
    x1 = halo1.quantities.get('particle_position_x') * u.cm
    y1 = halo1.quantities.get('particle_position_y') * u.cm
    z1 = halo1.quantities.get('particle_position_z') * u.cm
    radius1 = halo1.quantities.get('virial_radius') * u.cm
    
    # loop through second halo
    for halo2 in hc.halo_list:
        # find second index
        index2 = halo1.quantities.get('particle_identifier')
        
        if index2 <= index1:
            continue
        
        # find parameters of halo2
        x2 = halo2.quantities.get('particle_position_x') * u.cm
        y2 = halo2.quantities.get('particle_position_y') * u.cm
        z2 = halo2.quantities.get('particle_position_z') * u.cm
        radius2 = halo2.quantities.get('virial_radius') * u.cm
        
        # calculate dist between halo centers
        xdist = np.absolute(x2 - x1)
        ydist = np.absolute(y2 - y1)
        zdist = np.absolute(z2 - z1)
        netdist = (xdist**2 + ydist**2 + zdist**2)**0.5
        
        # if netdist < both radii, halo with smaller radius is satellite
        if netdist < radius1 and netdist < radius2:
            # check which radius smaller, if so, add index to list
            if radius2 < radius1:
                satellite.append(index2)
            else:
                satellite.append(index1)
                # set bool to true for later
                isSatellite = True
        
        # if netdist only < rad1, halo2 is satellite
        elif netdist < radius1:
            satellite.append(index2)
        
        # if netdist only < rad2, halo 1 is satellite
        elif netdist < radius2:
            satellite.append(index1)
            # set bool to true for later
            isSatellite = True
        
    # after loop, if satellite bool still false, and halo1 not in satellite
    # list, then add to non-satellite list
    if not isSatellite and index1 not in satellite:
        nonsatellite.append(index1)
    
    

In [81]:
testhalo = hc.halo_list.

<function list.index>

In [98]:
nonsatellite[6].value

array(6.0)

In [74]:
tempt1 = nonsatellite[3]
tempt2 = nonsatellite[4]

if tempt1 < tempt2:
    print('true')
else:
    print('false')

true


In [43]:
halo1 = hc.halo_list[2586]
halo2 = hc.halo_list[632]

# find first index
index1 = halo1.quantities.get('particle_identifier')

# find parameters of halo1
x1 = halo1.quantities.get('particle_position_x')
y1 = halo1.quantities.get('particle_position_y')
z1 = halo1.quantities.get('particle_position_z')
radius1 = halo1.quantities.get('virial_radius')

# find second index
index2 = halo1.quantities.get('particle_identifier')
        
# find parameters of halo2
x2 = halo2.quantities.get('particle_position_x')
y2 = halo2.quantities.get('particle_position_y')
z2 = halo2.quantities.get('particle_position_z')
radius2 = halo2.quantities.get('virial_radius')

# calculate dist between halo centers
xdist = np.absolute(x2 - x1)
ydist = np.absolute(y2 - y1)
zdist = np.absolute(z2 - z1)
netdist = (xdist**2 + ydist**2 + zdist**2)**0.5
        


In [44]:
print(netdist, '\n', radius1, '\n', radius2)

1.388177080219809e+24 cm 
 7.437515351055433e+21 cm 
 4.899934778698262e+21 cm


In [25]:
print(xdist, '\n', ydist, '\n', zdist)

3.1682811356454203e+24 cm 
 8.026487563195402e+23 cm 
 9.789640593412424e+23 cm


In [26]:
print(x1, '\n', x2, '\n', y1, '\n', y2)

1.1963596279206126e+25 cm 
 8.795315143560705e+24 cm 
 1.2021895291189851e+25 cm 
 1.121924653487031e+25 cm


In [27]:
halo2.quantities.get('particle_position_x').to('kpc')

2850.367516627491 kpc

In [152]:
# import basic libraries
import yt
import pickle
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

# import halo catalogue func
from yt.analysis_modules.halo_analysis.api import *

# import functions to ensure hc works
import tempfile
import shutil
import os

# Create temporary directory for storing files
tmpdir = tempfile.mkdtemp()

# load data dataset
# ds = yt.load('~/../../tigress/cen/LG4_2048_40pc/RD0070/redshift0070')

# load halo dataset
halos_ds = yt.load('./LG4_2048_40pc/rd0070/halo_catalogs/catalog/catalog0070_thres160.0.h5')

# Instantiate a catalog using those two paramter files
hc = HaloCatalog(halos_ds=halos_ds, output_dir=os.path.join(tmpdir, 'halo_catalog'))
hc.load()

# specify boundaries of zoom-in box
# scaling factor multiplied by info from text file
# units in cm
scaling = 2.22535525e+25 # scales dataset coords to cm

xmin = scaling*0.39319589 * u.cm
ymin = scaling*0.42984636 * u.cm
zmin = scaling*0.41706725 * u.cm

xmax = scaling*0.56298484 * u.cm
ymax = scaling*0.55089246 * u.cm
zmax = scaling*0.56698254 * u.cm

# create lists for satellites and non-satellites
satellite = []
nonsatellite = []

# fill up lists
for halo1 in hc.halo_list:
    # find first index
    index1 = halo1.quantities.get('particle_identifier')

    # create bool to check if this halo isn't satellite
    isSatellite = False

    # find parameters of halo1
    x1 = halo1.quantities.get('particle_position_x').in_units('cm')
    y1 = halo1.quantities.get('particle_position_y').in_units('cm')
    z1 = halo1.quantities.get('particle_position_z').in_units('cm')
    radius1 = halo1.quantities.get('virial_radius').in_units('cm')
    mass1 = halo1.quantities.get('particle_mass').in_units('Msun')

    # loop through second halo
    # checking if halo1 is satellite of halo2
    for halo2 in hc.halo_list:
        
        # if mass1 > mass2, skip other checks
        mass2 = halo2.quantities.get('particle_mass').in_units('Msun')
        if mass1 > mass2:
            continue
        
        # find second index
        index2 = halo1.quantities.get('particle_identifier')
        
        # skip tests if same halo
        if index1 == index2:
            continue

        # find parameters of halo2
        x2 = halo2.quantities.get('particle_position_x').in_units('cm')
        y2 = halo2.quantities.get('particle_position_y').in_units('cm')
        z2 = halo2.quantities.get('particle_position_z').in_units('cm')
        radius2 = halo2.quantities.get('virial_radius').in_units('cm')
        
        # calculate dist between halo centers
        xdist = np.absolute(x2 - x1)
        ydist = np.absolute(y2 - y1)
        zdist = np.absolute(z2 - z1)
        netdist = (xdist**2 + ydist**2 + zdist**2)**0.5
        
        # if netdist < radius2, halo1 must be satellite of halo2
        if netdist < radius2:
            satellite.append(index1)
            isSatellite = True

    # after loop, if satellite bool still false, and halo1 not in satellite
    # list, then add to non-satellite list
    if not isSatellite and index1 not in satellite:
        nonsatellite.append(index1)
        
    # output result
    print("Processed", index1, isSatellite)

# store lists in files
with open('satellite_list0070v2.txt', 'wb') as satellite_file:
    pickle.dump(satellite, satellite_file)

with open('satellite_non_list0070v2.txt', 'wb') as nonsatellite_file:
    pickle.dump(nonsatellite, nonsatellite_file)

yt : [INFO     ] 2017-06-30 12:31:34,824 Parameters: current_time              = 3.75573046409e+16
yt : [INFO     ] 2017-06-30 12:31:34,826 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-06-30 12:31:34,830 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-30 12:31:34,833 Parameters: domain_right_edge         = [  2.22535525e+25   2.22535525e+25   2.22535525e+25]
yt : [INFO     ] 2017-06-30 12:31:34,836 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-30 12:31:34,838 Parameters: current_redshift          = 4.99999998323
yt : [INFO     ] 2017-06-30 12:31:34,840 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-06-30 12:31:34,843 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-06-30 12:31:34,845 Parameters: hubble_constant           = 0.6933
yt : [INFO     ] 2017-06-30 12:31:34,854 Allocating for 9.178e+03 particles (index particle type 'all')
yt : [INFO     ] 2017-06-30 12:31:34,87

Processed 0.0 dimensionless False
Processed 1.0 dimensionless False
Processed 2.0 dimensionless False
Processed 3.0 dimensionless False
Processed 4.0 dimensionless False
Processed 5.0 dimensionless False
Processed 6.0 dimensionless False
Processed 7.0 dimensionless False
Processed 8.0 dimensionless False
Processed 9.0 dimensionless False
Processed 10.0 dimensionless False
Processed 11.0 dimensionless False
Processed 12.0 dimensionless False
Processed 13.0 dimensionless False
Processed 14.0 dimensionless False
Processed 15.0 dimensionless False
Processed 16.0 dimensionless False
Processed 17.0 dimensionless False
Processed 18.0 dimensionless False
Processed 19.0 dimensionless False
Processed 20.0 dimensionless False
Processed 21.0 dimensionless False
Processed 22.0 dimensionless False
Processed 23.0 dimensionless False
Processed 24.0 dimensionless False
Processed 25.0 dimensionless False
Processed 26.0 dimensionless False
Processed 27.0 dimensionless False
Processed 28.0 dimensionless F

Processed 230.0 dimensionless False
Processed 231.0 dimensionless False
Processed 232.0 dimensionless False
Processed 233.0 dimensionless False
Processed 234.0 dimensionless False
Processed 235.0 dimensionless False
Processed 236.0 dimensionless False
Processed 237.0 dimensionless False
Processed 238.0 dimensionless False
Processed 239.0 dimensionless False
Processed 240.0 dimensionless False
Processed 241.0 dimensionless False
Processed 242.0 dimensionless False
Processed 243.0 dimensionless False
Processed 244.0 dimensionless False
Processed 245.0 dimensionless False
Processed 246.0 dimensionless False
Processed 247.0 dimensionless False
Processed 248.0 dimensionless False
Processed 249.0 dimensionless False
Processed 250.0 dimensionless False
Processed 251.0 dimensionless False
Processed 252.0 dimensionless False
Processed 253.0 dimensionless False
Processed 254.0 dimensionless False
Processed 255.0 dimensionless False
Processed 256.0 dimensionless False
Processed 257.0 dimensionles

Processed 457.0 dimensionless False
Processed 458.0 dimensionless False
Processed 459.0 dimensionless False
Processed 460.0 dimensionless False
Processed 461.0 dimensionless False
Processed 462.0 dimensionless False
Processed 463.0 dimensionless False
Processed 464.0 dimensionless False
Processed 465.0 dimensionless False
Processed 466.0 dimensionless False
Processed 467.0 dimensionless False
Processed 468.0 dimensionless False
Processed 469.0 dimensionless False
Processed 470.0 dimensionless False
Processed 471.0 dimensionless False
Processed 472.0 dimensionless False
Processed 473.0 dimensionless False
Processed 474.0 dimensionless False
Processed 475.0 dimensionless False
Processed 476.0 dimensionless False
Processed 477.0 dimensionless False
Processed 478.0 dimensionless False
Processed 479.0 dimensionless False
Processed 480.0 dimensionless False
Processed 481.0 dimensionless False
Processed 482.0 dimensionless False
Processed 483.0 dimensionless False
Processed 484.0 dimensionles

Processed 684.0 dimensionless False
Processed 685.0 dimensionless False
Processed 686.0 dimensionless False
Processed 687.0 dimensionless False
Processed 688.0 dimensionless False
Processed 689.0 dimensionless False
Processed 690.0 dimensionless False
Processed 691.0 dimensionless False
Processed 692.0 dimensionless False
Processed 693.0 dimensionless False
Processed 694.0 dimensionless False
Processed 695.0 dimensionless False
Processed 696.0 dimensionless False
Processed 697.0 dimensionless False
Processed 698.0 dimensionless False
Processed 699.0 dimensionless False
Processed 700.0 dimensionless False
Processed 701.0 dimensionless False
Processed 702.0 dimensionless False
Processed 703.0 dimensionless False
Processed 704.0 dimensionless False
Processed 705.0 dimensionless False
Processed 706.0 dimensionless False
Processed 707.0 dimensionless False
Processed 708.0 dimensionless False
Processed 709.0 dimensionless False
Processed 710.0 dimensionless False
Processed 711.0 dimensionles

...

Processed 8206.0 dimensionless False
Processed 8207.0 dimensionless False
Processed 8208.0 dimensionless False
Processed 8209.0 dimensionless False
Processed 8210.0 dimensionless False
Processed 8211.0 dimensionless False
Processed 8212.0 dimensionless False
Processed 8213.0 dimensionless False
Processed 8214.0 dimensionless False
Processed 8215.0 dimensionless False
Processed 8216.0 dimensionless False
Processed 8217.0 dimensionless False
Processed 8218.0 dimensionless False
Processed 8219.0 dimensionless False
Processed 8220.0 dimensionless False
Processed 8221.0 dimensionless False
Processed 8222.0 dimensionless False
Processed 8223.0 dimensionless False
Processed 8224.0 dimensionless False
Processed 8225.0 dimensionless False
Processed 8226.0 dimensionless False
Processed 8227.0 dimensionless False
Processed 8228.0 dimensionless False
Processed 8229.0 dimensionless False
Processed 8230.0 dimensionless False
Processed 8231.0 dimensionless False
Processed 8232.0 dimensionless False
P

Processed 8427.0 dimensionless False
Processed 8428.0 dimensionless False
Processed 8429.0 dimensionless False
Processed 8430.0 dimensionless False
Processed 8431.0 dimensionless False
Processed 8432.0 dimensionless False
Processed 8433.0 dimensionless False
Processed 8434.0 dimensionless False
Processed 8435.0 dimensionless False
Processed 8436.0 dimensionless False
Processed 8437.0 dimensionless False
Processed 8438.0 dimensionless False
Processed 8439.0 dimensionless False
Processed 8440.0 dimensionless False
Processed 8441.0 dimensionless False
Processed 8442.0 dimensionless False
Processed 8443.0 dimensionless False
Processed 8444.0 dimensionless False
Processed 8445.0 dimensionless False
Processed 8446.0 dimensionless False
Processed 8447.0 dimensionless False
Processed 8448.0 dimensionless False
Processed 8449.0 dimensionless False
Processed 8450.0 dimensionless False
Processed 8451.0 dimensionless False
Processed 8452.0 dimensionless False
Processed 8453.0 dimensionless False
P

Processed 8648.0 dimensionless False
Processed 8649.0 dimensionless False
Processed 8650.0 dimensionless False
Processed 8651.0 dimensionless False
Processed 8652.0 dimensionless False
Processed 8653.0 dimensionless False
Processed 8654.0 dimensionless False
Processed 8655.0 dimensionless False
Processed 8656.0 dimensionless False
Processed 8657.0 dimensionless False
Processed 8658.0 dimensionless False
Processed 8659.0 dimensionless False
Processed 8660.0 dimensionless False
Processed 8661.0 dimensionless False
Processed 8662.0 dimensionless False
Processed 8663.0 dimensionless False
Processed 8664.0 dimensionless False
Processed 8665.0 dimensionless False
Processed 8666.0 dimensionless False
Processed 8667.0 dimensionless False
Processed 8668.0 dimensionless False
Processed 8669.0 dimensionless False
Processed 8670.0 dimensionless False
Processed 8671.0 dimensionless False
Processed 8672.0 dimensionless False
Processed 8673.0 dimensionless False
Processed 8674.0 dimensionless False
P

Processed 8869.0 dimensionless False
Processed 8870.0 dimensionless False
Processed 8871.0 dimensionless False
Processed 8872.0 dimensionless False
Processed 8873.0 dimensionless False
Processed 8874.0 dimensionless False
Processed 8875.0 dimensionless False
Processed 8876.0 dimensionless False
Processed 8877.0 dimensionless False
Processed 8878.0 dimensionless False
Processed 8879.0 dimensionless False
Processed 8880.0 dimensionless False
Processed 8881.0 dimensionless False
Processed 8882.0 dimensionless False
Processed 8883.0 dimensionless False
Processed 8884.0 dimensionless False
Processed 8885.0 dimensionless False
Processed 8886.0 dimensionless False
Processed 8887.0 dimensionless False
Processed 8888.0 dimensionless False
Processed 8889.0 dimensionless False
Processed 8890.0 dimensionless False
Processed 8891.0 dimensionless False
Processed 8892.0 dimensionless False
Processed 8893.0 dimensionless False
Processed 8894.0 dimensionless False
Processed 8895.0 dimensionless False
P

Processed 9090.0 dimensionless False
Processed 9091.0 dimensionless False
Processed 9092.0 dimensionless False
Processed 9093.0 dimensionless False
Processed 9094.0 dimensionless False
Processed 9095.0 dimensionless False
Processed 9096.0 dimensionless False
Processed 9097.0 dimensionless False
Processed 9098.0 dimensionless False
Processed 9099.0 dimensionless False
Processed 9100.0 dimensionless False
Processed 9101.0 dimensionless False
Processed 9102.0 dimensionless False
Processed 9103.0 dimensionless False
Processed 9104.0 dimensionless False
Processed 9105.0 dimensionless False
Processed 9106.0 dimensionless False
Processed 9107.0 dimensionless False
Processed 9108.0 dimensionless False
Processed 9109.0 dimensionless False
Processed 9110.0 dimensionless False
Processed 9111.0 dimensionless False
Processed 9112.0 dimensionless False
Processed 9113.0 dimensionless False
Processed 9114.0 dimensionless False
Processed 9115.0 dimensionless False
Processed 9116.0 dimensionless False
P

In [153]:
satellite

[]

In [155]:
nonsatellite[-1]

9177.0 dimensionless